<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
LIGHT TABLE - GENERATION</p><br>

This notebook generate Light Tables based on nexus full-simulations of light, generating a DataFrame that should be used by DetSim.

TAREAS POR IMPLEMENTAR:

* Descripcion de lo que hace cada una de las funciones y sus parametros
* Leer el setup de un fichero de configuracion
* Anyadir tests
* Posibilidad de muestrear los volumenes / areas con un patron asimetrico (mas denso a radios altos, para reflejar mejor los cambios que se producen cerca de los bordes).
* Hacer el setting de IC y de NEXUS desde las funciones, no desde el terminal
* Anyadir tests de que las simulaciones han corrido bien en las colas.

In [1]:
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [26]:
# General importings
import os
import numpy  as np
import pandas as pd

from typing import Tuple
from math   import ceil

# Specific IC stuff
import invisible_cities.core.system_of_units  as units
from invisible_cities.io.mcinfo_io        import load_mcsensor_response_df
from invisible_cities.io.mcinfo_io        import get_sensor_types

# Light Table stuff
from sim_functions       import make_init_file
from sim_functions       import make_config_file
from sim_functions       import run_sim
from sim_functions       import get_num_photons

from table_functions     import get_table_positions
from table_functions     import get_working_paths
from table_functions     import build_table
from table_functions     import get_fnames
from table_functions     import get_table_fname
from table_functions     import get_detector_dimensions

In [3]:
%load_ext autoreload
%autoreload 2

# SETUP DATA

In [4]:
VERBOSITY = True

In [5]:
MAX_PHOTONS_PER_EVT = 100000

In [6]:
### Current options: "NEXT_NEW", "NEXT100", "NEXT_FLEX"
det_name = "NEXT_NEW"


### Type of Light Table: energy or tracking
table_type = "tracking"


### Signal Type: S1 or S2
signal_type = "S2"


### Sensor name.
# Tipically PmtR11410 for energy tables and SiPM for tracking tables. 
#sensor_name = "PmtR11410"
sensor_name = "SiPM"


### Table pitch
#pitch = (200.0 * units.mm, 200.0 * units.mm, 200.0 * units.mm)
#pitch = (20.0 * units.mm, 20.0 * units.mm, 40.0 * units.mm)
pitch = (30.0 * units.mm, 30.0 * units.mm, 1.0 * units.mm)
#pitch = (1.0 * units.mm, 1.0 * units.mm, 1.0 * units.mm)


### Table num photons / point
photons_per_point = 100000


### Getting (photons / point) & (events / point) & (photons / event)
events_per_point  = 1
photons_per_event = photons_per_point

if photons_per_event > MAX_PHOTONS_PER_EVT:
    events_per_point  = ceil(photons_per_point / MAX_PHOTONS_PER_EVT)
    photons_per_event = MAX_PHOTONS_PER_EVT
    photons_per_point = events_per_point * photons_per_event

    
### Verbosity
if VERBOSITY:
    print(f"***** Generating {det_name} Light Table  *****\n")
    print(f"*** Type: {table_type}  -  Signal: {signal_type}  -  Sensor: {sensor_name}")
    print(f"*** Pitch: {pitch} mm")
    print(f"*** Photons/Point = {photons_per_point:.1e} splitted into ...")
    print(f"*** {events_per_point} Events/Point * {photons_per_event:.1e} Photons/Event")

***** Generating NEXT_NEW Light Table  *****

*** Type: tracking  -  Signal: S2  -  Sensor: SiPM
*** Pitch: (30.0, 30.0, 1.0) mm
*** Photons/Point = 1.0e+05 splitted into ...
*** 1 Events/Point * 1.0e+05 Photons/Event


In [7]:
table_positions = get_table_positions(det_name, table_type, signal_type, pitch)

# Vervosity
if VERBOSITY:
    print(f"*** Total number of points: {len(table_positions)}")
    print(table_positions)

*** Total number of points: 30
[(25.0, 25.0, -0.5), (25.0, 25.0, -1.5), (25.0, 25.0, -2.5), (25.0, 25.0, -3.5), (25.0, 25.0, -4.5), (25.0, 25.0, -5.5), (55.0, 25.0, -0.5), (55.0, 25.0, -1.5), (55.0, 25.0, -2.5), (55.0, 25.0, -3.5), (55.0, 25.0, -4.5), (55.0, 25.0, -5.5), (85.0, 25.0, -0.5), (85.0, 25.0, -1.5), (85.0, 25.0, -2.5), (85.0, 25.0, -3.5), (85.0, 25.0, -4.5), (85.0, 25.0, -5.5), (115.0, 25.0, -0.5), (115.0, 25.0, -1.5), (115.0, 25.0, -2.5), (115.0, 25.0, -3.5), (115.0, 25.0, -4.5), (115.0, 25.0, -5.5), (145.0, 25.0, -0.5), (145.0, 25.0, -1.5), (145.0, 25.0, -2.5), (145.0, 25.0, -3.5), (145.0, 25.0, -4.5), (145.0, 25.0, -5.5)]


In [8]:
# Getting PATHS
config_path, log_path, dst_path, table_path = get_working_paths(det_name)

# Verbosity
if VERBOSITY:
    print(f"*** Config PATH: {config_path}")
    print(f"*** Log    PATH: {log_path}")
    print(f"*** Dst    PATH: {dst_path}")
    print(f"*** Table  PATH: {table_path}")

*** Config PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/config/
*** Log    PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/log/
*** Dst    PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/dst/
*** Table  PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/table/


# LIGHT SIMULATIONS

In [9]:
for pos in table_positions:
    
    # file names
    init_fname, config_fname, log_fname, dst_fname = get_fnames(det_name ,pos)

    # make configuration files
    make_init_file(det_name, init_fname, config_fname)
    
    make_config_file(det_name, config_fname, dst_fname,
                     pos[0], pos[1], pos[2],
                     photons_per_event)
    
    # Runing the simulation
    if VERBOSITY:
        print(f"* Runing {det_name} sim of {photons_per_point:.1e} photons from {pos} ...")
        
    # Check if the sim is already run with the correct num_photons
    if os.path.isfile(dst_fname + '.h5'):
        run_photons = get_num_photons(dst_fname + '.h5')
        if (get_num_photons(dst_fname + '.h5') < photons_per_point):
            print("  Simulation already run previously with less events, so re-running ...")
            run_sim(init_fname, log_fname, events_per_point)
        else:
            print("  Simulation already run previously, so skipping ...")
    
    else:
        run_sim(init_fname, log_fname, events_per_point)

* Runing NEXT_NEW sim of 1.0e+05 photons from (25.0, 25.0, -0.5) ...
  Simulation already run previously, so skipping ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (25.0, 25.0, -1.5) ...
  Simulation already run previously, so skipping ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (25.0, 25.0, -2.5) ...
  Simulation already run previously, so skipping ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (25.0, 25.0, -3.5) ...
  Simulation already run previously, so skipping ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (25.0, 25.0, -4.5) ...
  Simulation already run previously, so skipping ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (25.0, 25.0, -5.5) ...
  Simulation already run previously, so skipping ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (55.0, 25.0, -0.5) ...
  Simulation already run previously, so skipping ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (55.0, 25.0, -1.5) ...
  Simulation already run previously, so skipping ...
* Runing NEXT_NEW sim of

# TABLE GENERATION

In [10]:
light_table = build_table(det_name, table_type, signal_type, sensor_name, pitch)


* Getting data from NEXT_NEW - Distance: (0.0, -1.0) ...
  /Users/Javi/Development/NextLightTable/data/NEXT_NEW/dst/NEXT_NEW.x_25.0.y_25.0.z_-0.5.next.h5
  Simulation run with   1000000 initial photons.
  Charge:     37 -> Sensor prob: 3.700e-05

* Getting data from NEXT_NEW - Distance: (0.0, -2.0) ...
  /Users/Javi/Development/NextLightTable/data/NEXT_NEW/dst/NEXT_NEW.x_25.0.y_25.0.z_-1.5.next.h5
  Simulation run with   1000000 initial photons.
  Charge:     49 -> Sensor prob: 4.900e-05

* Getting data from NEXT_NEW - Distance: (0.0, -3.0) ...
  /Users/Javi/Development/NextLightTable/data/NEXT_NEW/dst/NEXT_NEW.x_25.0.y_25.0.z_-2.5.next.h5
  Simulation run with   1000000 initial photons.
  Charge:     74 -> Sensor prob: 7.400e-05

* Getting data from NEXT_NEW - Distance: (0.0, -4.0) ...
  /Users/Javi/Development/NextLightTable/data/NEXT_NEW/dst/NEXT_NEW.x_25.0.y_25.0.z_-3.5.next.h5
  Simulation run with   1000000 initial photons.
  Charge:     95 -> Sensor prob: 9.500e-05

* Getting d

In [11]:
light_table.head()

,z_m1,z_m2,z_m3,z_m4,z_m5,z_m6
dist_xy,,,,,,
0.0,0.000037,0.000049,0.000074,0.000095,0.000121,0.000186
30.0,0.000003,0.000000,0.000001,0.000002,0.000001,0.000000
60.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000001
90.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
120.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
# Storing the DataFrame
light_table_fname = table_path + get_table_fname(det_name, table_type,
                                                 signal_type, sensor_name)

if VERBOSITY:
    print(f"\n*** Storing Light Table in {light_table_fname} ...")

light_table.to_hdf(light_table_fname, '/LightTable', mode   = 'w',
                   format = 'table', data_columns = True)


*** Storing Light Table in /Users/Javi/Development/NextLightTable/data/NEXT_NEW/table/NEXT_NEW.tracking.S2.SiPM.LightTable.h5 ...


In [30]:
# Adding a table with config info

dimensions = get_detector_dimensions(det_name)

config_columns =  ['parameter', 'value']

config_data    = [['detector' ,            det_name],
                  ['ACTIVE_rad',           str(dimensions['ACTIVE_radius'])],
                  ['ACTIVE_length',        str(dimensions['ACTIVE_length'])],
                  ['EL_GAP',               str(dimensions['EL_gap'])],
                  ['reference_sensor_id',  str(dimensions['ref_sensor'][0])],
                  ['table_type',           table_type],
                  ['signal_type',          signal_type],
                  ['sensor',               sensor_name],
                  ['pitch_x',              str(pitch[0])],
                  ['pitch_y',              str(pitch[1])],
                  ['pitch_z',              str(pitch[2])],
                  ['photons_per_point',    str(photons_per_point)],
                  ['photons_per_event',    str(photons_per_event)],
                  ['events_per_point',     str(events_per_point)],
                  ['total_points',         str(len(table_positions))],
                  ['table_path',           table_path],
                  ['dst_path',             dst_path],
                  ['config_path',          config_path],
                  ['log_path',             log_path]]

config_table = pd.DataFrame(config_data, columns = config_columns)
config_table.set_index("parameter", inplace = True)

In [31]:
config_table

,value
parameter,
detector,NEXT_NEW
ACTIVE_rad,208.0
ACTIVE_length,532.0
EL_GAP,6.0
reference_sensor,17018
table_type,tracking
signal_type,S2
sensor,SiPM
pitch_x,30.0


In [24]:
config_table.to_hdf(light_table_fname, '/Config', mode   = 'a',
                    format = 'table', data_columns = True)